In [92]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time  

#kbo사이트에서 타자들의 기록을 크롤링하고 싶었습니다. 웹 크롤링 & 데이터 분석 with 파이썬 책을 참고해 기본적인 코드를 짰습니다.
#제일 어려웠던건 다음페이지로 넘어가는 버튼을 클릭하는 코드를 짜는 것이었습니다.
#여러 블로그와 유튜브를 참고하여 버튼을 클릭하는 코드를 만들었습니다.
#그 후에 코드를 실행했을때 계속 1페이지의 내용이 중복되어 출력되었습니다.
#해결방법으로 반복문을 사용했을때 탈출 조건을 어떻게 짜야할지 몰라 제대로 활용하지 못했습니다.
#비슷한 사이트들의 다른 코드들을 보며 공부했는데 거기서 다른점을 찾아보니 페이지 이동후 충분한 시간을 기다리지 않아 발생한 문제였습니다.
#그래서 최종적으로 선수들의 기록이 출력되고 동시에 엑셀로 저장되는 코드를 완성했습니다.

In [93]:
driver = webdriver.Chrome() #WebDriver(Selenium 라이브러리 사용해 웹 브라우저 자동화하는 데 사용) 초기화 부분

#WebDriver 사용해, 지정된 주소로 이동
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/BasicOld.aspx?sort=HRA_RT') 


In [94]:
#와일드카드 결정전 타율 -> 정규시즌 타율 페이지로 넘어가도록 버튼 클릭

button = driver.find_element(By.XPATH, '/html/body/form/div[3]/section/div/div/div[2]/div[3]/div[1]/div[2]/select')
button.click()
time.sleep(2) 

button = driver.find_element(By.XPATH, '/html/body/form/div[3]/section/div/div/div[2]/div[3]/div[1]/div[2]/select/option[1]')
button.click()
time.sleep(2) 


In [96]:
list_data = [] #추출하는 데이터 저장하기 위해 비어있는 리스트 초기화

#Selenium 사용해 웹 페이지의 소스 코드를 가져와 BeautifulSoup를 사용해서 파싱
bs_obj = BeautifulSoup(driver.page_source, "html.parser") 

In [97]:
#BeautifulSoup를 사용해, HTML 웹 페이지에서 테이블 데이터를 추출
#해당 테이블의 열 제목 column_names 리스트에 저장

table_tag = bs_obj.find_all("table")[0] #<table> 요소를 찾아, 첫 번째 테이블을 선택하여 table_tag 변수에 저장
thead = table_tag.find("thead") #<thead> 요소를 찾아, 해당 요소 thead 변수에 저장
column_names = [th.text.strip() for th in thead.find_all("th")] # thead 요소에서 모든 <th> 요소를 찾아 column_names 리스트에 저장.  
                                                                # .text 속성을 사용하여 각 <th> 요소의 텍스트 내용 추출. 
                                                                # .strip() 메서드를 사용해 좌우 공백 제거


In [98]:
# HTML 웹 페이지에서 모든 <tr> 요소 찾아 player 변수에 저장
# player_data_list라는 빈 리스트 초기화

player = bs_obj.find_all("tr")

player_data_list = [] #나중에 각 행에서 추출한 데이터를 저장하는 데 사용

In [100]:
#BeautifulSoup를 사용해 웹 페이지에서 추출한 데이터를 처리하고 출력

print(column_names) 


for idx, element in enumerate(player[1:]): #player 리스트에서 각 행을 반복적으로 처리. 
                        #enumerate 함수를 사용해 각 행의 인덱스와 해당 행의 내용을 반복
    #각 행에서 <td> 요소 찾아 해당 셀의 내용을 추출, 각 셀의 내용 player_data 리스트에 저장
    player_data = [td.text.strip() for td in element.find_all("td")] 
    player_data_list.append(player_data) #player_data 리스트 player_data_list 리스트에 추가
    
    print(player_data)

['순위', '선수명', '팀명', 'AVG', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SAC', 'SF']
['1', '손아섭', 'NC', '0.339', '140', '609', '551', '97', '187', '36', '3', '5', '244', '65', '1', '5']
['2', '구자욱', '삼성', '0.336', '119', '515', '453', '65', '152', '37', '1', '11', '224', '71', '2', '3']
['3', '김혜성', '키움', '0.335', '137', '621', '556', '104', '186', '29', '6', '7', '248', '57', '0', '5']
['4', '홍창기', 'LG', '0.332', '141', '643', '524', '109', '174', '35', '2', '1', '216', '65', '3', '6']
['5', '에레디아', 'SSG', '0.323', '122', '523', '473', '76', '153', '29', '0', '12', '218', '76', '1', '1']
['6', '김선빈', 'KIA', '0.320', '119', '473', '419', '41', '134', '16', '0', '0', '150', '48', '8', '3']
['7', '박건우', 'NC', '0.319', '130', '533', '458', '70', '146', '34', '2', '12', '220', '85', '1', '9']
['8', '박민우', 'NC', '0.316', '124', '509', '452', '76', '143', '20', '7', '2', '183', '46', '5', '3']
['9', '오스틴', 'LG', '0.313', '139', '583', '520', '87', '163', '29', '4', '23', '269', 

In [101]:

df = pd.DataFrame(player_data_list, columns=column_names) #player_data_list에 저장된 데이터를 DataFrame으로 변환

df.to_excel("hitter_data.xlsx", index=False) #hitter_data.xlsx 엑셀파일로 저장

driver.quit() #webdriver 종료

In [102]:
import bs4

In [103]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [104]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
